# Chapter 5 - Dive into deep learning

this is assignment on class introduction to deep learning 

source : [Chapter 5 - Deep Learning Computation](https://d2l.ai/chapter_deep-learning-computation/index.html)

In [1]:
import tensorflow as tf

# 5.1 Layers and Blocks

konsep blok jaringan saraf. Sebuah blok dapat menggambarkan satu lapisan, komponen yang terdiri dari beberapa lapisan, atau seluruh model itu sendiri! Salah satu manfaat bekerja dengan abstraksi blok adalah mereka dapat digabungkan menjadi artefak yang lebih besar, seringkali secara rekursif

In [2]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.15503676,  0.3031974 , -0.16672961, -0.06071816, -0.35859424,
        -0.27953333, -0.12770565, -0.28128797, -0.16478902, -0.15558939],
       [ 0.01638052,  0.10633644, -0.0843289 ,  0.0599676 , -0.29568592,
        -0.23891413, -0.08447594, -0.1988513 , -0.05017764, -0.01582683]],
      dtype=float32)>

## 5.1.1 Custom Block

1. Serap data input sebagai argumen untuk fungsi *forward propagation*
2. Hasilkan output dengan membuat fungsi *forward propagation* mengembalikan nilai. Perhatikan bahwa output mungkin memiliki bentuk yang berbeda dari input. Misalnya, lapisan pertama yang terhubung sepenuhnya dalam model kami di atas menyerap input dimensi arbitrer tetapi mengembalikan output dimensi 256.
3. Hitung gradien outputnya sehubungan dengan inputnya, yang dapat diakses melalui fungsi *backpropagation*. Biasanya ini terjadi secara otomatis.
4. Menyimpan dan menyediakan akses ke parameter yang diperlukan untuk menjalankan komputasi propagasi maju.

5. Inisialisasi parameter model sesuai kebutuhan.



In [3]:
class MLP(tf.keras.Model):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Model` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        # Hidden layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def call(self, X):
        return self.out(self.hidden((X)))

In [4]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.12097374, -0.14194793,  0.11305366, -0.39977127, -0.28660795,
        -0.2520434 , -0.00121207, -0.24595582, -0.2936792 , -0.20400342],
       [ 0.13113678, -0.13972023,  0.21779458, -0.32140923, -0.03125362,
        -0.2588921 , -0.00937137, -0.19849241, -0.09207319, -0.15827566]],
      dtype=float32)>

## 5.1.2 Sequential Block

In [5]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, `block` is an instance of a `tf.keras.layers.Layer`
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [6]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.38855797, -0.25349426,  0.11016544, -0.30879498, -0.34653088,
        -0.47085696, -0.19308163,  0.11986673, -0.43464637,  0.25295413],
       [-0.47622517, -0.00277524,  0.06996723, -0.2818883 , -0.43247095,
        -0.44782716, -0.20436361, -0.09941158, -0.43303567,  0.34176326]],
      dtype=float32)>

## 5.1.3 Executing code in **Forward Propagation** Function 

Katakanlah misalnya bahwa kita menginginkan lapisan yang menghitung fungsi
$f(x, w) = c ⋅ w^⊤ x$, dimana 

$x$ merupakan input, 

$w$ adalah parameternya, dan 

$c$ adalah beberapa konstanta tertentu yang tidak diperbarui selama pengoptimalan


In [7]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # Random weight parameters created with `tf.constant` are not updated
        # during training (i.e., constant parameters)
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # Use the created constant parameters, as well as the `relu` and
        # `matmul` functions
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.dense(X)
        # Control flow
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [8]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5712446>

In [9]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.97772586>

# 5.2 Parameters Management

In [10]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.04582323],
       [0.12907463]], dtype=float32)>

## 5.2.1 Parameters Access

In [11]:
print(net.layers[2].weights)

[<tf.Variable 'dense_13/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.58155143],
       [-0.9833052 ],
       [-0.66982424],
       [-0.18462217]], dtype=float32)>, <tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [12]:
# Targeted Parameters
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


In [13]:
# All Parameters at Once
print(net.layers[1].weights)
print(net.get_weights())

net.get_weights()[1]

[<tf.Variable 'dense_12/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.767579  , -0.46061572, -0.15807164, -0.840176  ],
       [ 0.8020235 , -0.23930198, -0.5298283 , -0.27171278],
       [-0.7094276 , -0.2226699 ,  0.03363997, -0.25592893],
       [-0.3977085 , -0.5417814 , -0.09114248,  0.29346186]],
      dtype=float32)>, <tf.Variable 'dense_12/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[ 0.767579  , -0.46061572, -0.15807164, -0.840176  ],
       [ 0.8020235 , -0.23930198, -0.5298283 , -0.27171278],
       [-0.7094276 , -0.2226699 ,  0.03363997, -0.25592893],
       [-0.3977085 , -0.5417814 , -0.09114248,  0.29346186]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 0.58155143],
       [-0.9833052 ],
       [-0.66982424],
       [-0.18462217]], dtype=float32), array([0.], dtype=float32)]


array([0., 0., 0., 0.], dtype=float32)

In [14]:
# Collecting Parameters from Nested Blocks
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

print(rgnet.summary()) # Print summary

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (2, 4)                    80        
                                                                 
 dense_18 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


## 5.2.2 Parameter Initialization

### 5.2.2.1 Built-in Initialization

In [15]:
# Built-in Initialization with
# Gaussian random variables with standard deviation 0.01

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_19/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.00455257, -0.00544824, -0.00103761, -0.00367225],
        [-0.01094247, -0.0053754 , -0.00108208,  0.01311664],
        [ 0.00711042, -0.01007599, -0.00038948,  0.00486159],
        [ 0.01340271, -0.00191556, -0.0080068 , -0.00547056]],
       dtype=float32)>,
 <tf.Variable 'dense_19/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [16]:
# Built-in Initialization with
# with constant value (say, 1)

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_21/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [17]:
# Built-in Initialization with
#   1. first layer with the Xavier initializer
#   2. second layer with constant value of 42.

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_23/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.08179694,  0.7055431 , -0.57094216, -0.10288644],
       [ 0.12294793, -0.5841122 ,  0.0385046 ,  0.7406489 ],
       [-0.20279175,  0.30120367, -0.44154307,  0.340581  ],
       [ 0.28249007,  0.7263681 ,  0.35400862, -0.75482   ]],
      dtype=float32)>
<tf.Variable 'dense_24/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


### 5.2.2.2 Custom Initialization

contoh dibawah ini kita ingin mendefinisikan penginisialisasi untuk parameter bobot $w$ menggunakan distribusi berikut :

$
\begin{split}\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}\end{split}
$

In [18]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.       ,  7.1431255, -0.       ,  8.979313 ],
       [-0.       , -5.6240654,  0.       , -7.999475 ],
       [-0.       , -0.       ,  0.       ,  0.       ],
       [ 8.706886 , -0.       , -7.641082 ,  9.704243 ]], dtype=float32)>


In [19]:
# setting parameters directly

net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  8.143126 ,  1.       ,  9.979313 ],
       [ 1.       , -4.6240654,  1.       , -6.999475 ],
       [ 1.       ,  1.       ,  1.       ,  1.       ],
       [ 9.706886 ,  1.       , -6.641082 , 10.704243 ]], dtype=float32)>

## 5.2.3 Tied Parameters

In [20]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


# 5.3 Deferred Initialization

Sejauh ini, sepertinya kami tidak ceroboh dalam menyiapkan jaringan kami. Secara khusus, kami melakukan hal-hal tidak intuitif berikut, yang mungkin tidak terlihat seperti seharusnya. 

Anda mungkin terkejut bahwa kode kami berjalan sama sekali. Lagi pula, tidak mungkin kerangka kerja pembelajaran yang mendalam dapat memberi tahu seperti apa dimensi input dari suatu jaringan. Triknya di sini adalah bahwa kerangka kerja menunda inisialisasi, menunggu hingga pertama kali kami melewatkan data melalui model, untuk menyimpulkan ukuran setiap lapisan dengan cepat.

### 5.3.1. Instantiating a Network

In [21]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [22]:
# pass data through the network to make the framework finally initialize parameters.
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

## 5.4 Custom Layers

Deep Learning menjadi populer karena ketersediaan berbagai *layers* yang dapat disusun dengan cara kreatif untuk merancang arsitektur yang cocok untuk berbagai tugas. Misalnya, para peneliti telah menemukan *layers* khusus untuk menangani gambar, teks, mengulang data sekuensial, dan melakukan pemrograman dinamis

### 5.4.1. Layers without Parameters

In [23]:
# layer that does not have any parameters

class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [24]:
# Fedding data to layer
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [25]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [ ]:
# send random data through the network and check that the mean is in fact 0

Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

### 5.4.2. Layers with Parameters

sekarang buat layer yang bisa mengatur akses, inisialisasi, berbagi, menyimpan, dan memuat parameter model

In [27]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [28]:
# instantiate the MyDense class and access its model parameters

dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[-0.01677377, -0.03413559,  0.08295175],
        [ 0.12640269, -0.04505158,  0.07163015],
        [ 0.04286781,  0.03438101, -0.06119176],
        [-0.04010909,  0.06591933, -0.07513057],
        [ 0.02709937,  0.05705613, -0.01169976]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [29]:
# forward propagation calculations using custom layers

dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.10330303, 0.02259478, 0.        ],
       [0.10658844, 0.        , 0.02003217]], dtype=float32)>

In [30]:
# construct models using custom layers

net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.02576924],
       [0.01531849]], dtype=float32)>

## 5.5. File I/O

saat menjalankan proses pelatihan yang panjang, praktik terbaik adalah menyimpan hasil antara (*checkpointing*) secara berkala untuk memastikan bahwa kami tidak kehilangan perhitungan beberapa hari jika kami tersandung kabel daya server kami. Jadi inilah saatnya untuk mempelajari cara memuat dan menyimpan vektor bobot individu dan seluruh model

### 5.5.1. Loading and Saving Tensors

Untuk tensor individual, kita dapat langsung menjalankan fungsi load dan save untuk membaca dan menulisnya masing-masing

In [31]:
import numpy as np
import tensorflow as tf

x = tf.range(4)
np.save('x-file.npy', x)

In [32]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [33]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

### 5.5.2. Loading and Saving Model Parameters

Menyimpan vektor bobot individu (atau tensor lainnya) berguna, tetapi akan sangat membosankan jika kita ingin menyimpan (dan kemudian memuat) seluruh model. Bagaimanapun, kita mungkin memiliki ratusan grup parameter yang tersebar di seluruh. Untuk alasan ini, kerangka kerja pembelajaran mendalam menyediakan fungsionalitas bawaan untuk memuat dan menyimpan seluruh jaringan

In [34]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [35]:
net.save_weights('mlp.params')

In [36]:
clone = MLP()
clone.load_weights('mlp.params')

In [37]:
clone = MLP()
clone.load_weights('mlp.params')

## 5.6. GPUs

In [38]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



### 5.6.1. Computing Devices

menentukan perangkat, seperti CPU dan GPU, untuk penyimpanan dan penghitungan. Secara default, tensor dibuat di memori utama dan kemudian menggunakan CPU untuk menghitungnya.


In [39]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7fa4eb1dad20>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7fa4eb0092d0>)

In [40]:
# # query the number of available GPUs

len(tf.config.experimental.list_physical_devices('GPU'))

0

In [42]:
# define two convenient functions that allow us to run code even if the requested GPUs do not exist

def try_gpu(i=0):  #save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():  #save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7fa4eb1e94b0>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x7fa4eb1e91e0>])

### 5.6.2. Tensors and GPUs

tensors are created on the CPU. We can query the device where the tensor is located.



In [43]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

#### 5.6.2.1. Storage on the GPU

kita dapat menentukan perangkat penyimpanan saat membuat tensor. Selanjutnya, kita membuat variabel tensor X pada GPU pertama


In [44]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

Assuming that you have at least two GPUs, the following code will create a random tensor on the second GPU.



In [45]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [49]:
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.40587854, 0.13060927, 0.47710764],
       [0.24682248, 0.21854162, 0.69264865]], dtype=float32)>

#### 5.6.2.2. Copying

In [50]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [51]:
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.4058785, 1.1306093, 1.4771076],
       [1.2468225, 1.2185416, 1.6926486]], dtype=float32)>

In [52]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

### 5.6.3. Neural Networks and GPUs

In [53]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [54]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.9220355],
       [0.9220355]], dtype=float32)>

In [55]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:CPU:0',
 '/job:localhost/replica:0/task:0/device:CPU:0')